### All needed imports

In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
assert tf.__version__.startswith('2')
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dense, GlobalAveragePooling2D, Dropout


### Limit GPU Usage

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only allocate 2GB of memory on the first GPU
    try:
        tf.config.set_logical_device_configuration(gpus[0], [tf.config.LogicalDeviceConfiguration(memory_limit=2048)])
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


2023-08-10 13:38:40.839413: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-08-10 13:38:40.934816: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-08-10 13:38:40.934962: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-08-10 13:38:40.973099: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

### Preprocessing Data for Model 1

#### Video Paths

In [3]:
falling_paths = [
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall2.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall3.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall4.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall1.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall5.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall6.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall7.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall8.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall9.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall10.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall11.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall12.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall13.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall14.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall15.mp4"
                 ]

default_paths = [
                 "./../datasets/vids/splitted/new_still/resized_logitech-default1.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default2.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default3.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default4.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default5.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default6.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default7.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default8.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default9.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default10.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default11.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default12.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default13.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default14.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default15.mp4"
                 ]

test_falling_paths = [
                "./../datasets/vids/testdata/moving/resized-test-fall1.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall2.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall3.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall4.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall5.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall6.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall7.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall8.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall9.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall10.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall11.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall12.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall13.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall14.mp4",
                "./../datasets/vids/testdata/moving/resized-test-fall15.mp4"
]

test_default_paths = [
                "./../datasets/vids/testdata/still/resized-test-default1.mp4",
                "./../datasets/vids/testdata/still/resized-test-default2.mp4",
                "./../datasets/vids/testdata/still/resized-test-default3.mp4",
                "./../datasets/vids/testdata/still/resized-test-default4.mp4",
                "./../datasets/vids/testdata/still/resized-test-default5.mp4",
                "./../datasets/vids/testdata/still/resized-test-default6.mp4",
                "./../datasets/vids/testdata/still/resized-test-default7.mp4",
                "./../datasets/vids/testdata/still/resized-test-default8.mp4",
                "./../datasets/vids/testdata/still/resized-test-default9.mp4",
                "./../datasets/vids/testdata/still/resized-test-default10.mp4",
                "./../datasets/vids/testdata/still/resized-test-default11.mp4",
                "./../datasets/vids/testdata/still/resized-test-default12.mp4",
                "./../datasets/vids/testdata/still/resized-test-default13.mp4",
                "./../datasets/vids/testdata/still/resized-test-default14.mp4",
                "./../datasets/vids/testdata/still/resized-test-default15.mp4"
]

#### Processing code, returns frame (num_frames, 224,224,3) and frame_diffs (num_frames, 224,224)

In [4]:
# Code to apply the same random transformation 

In [4]:
def process_videos(video_paths, label):
    frames = []
    frame_diffs = []
    labels = []
    
    for path in video_paths:
        video_cap = cv2.VideoCapture(path)
        
        prev_gray_frame = None
        
        while video_cap.isOpened():
            ret, frame = video_cap.read()
            
            if not ret:
                break
            
            # Resize and convert frame to RGB
            frame_resized = cv2.resize(frame, (224, 224))
            frame_rgb = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2RGB)
            
            # Convert frame to grayscale
            gray_frame = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2GRAY)

            if prev_gray_frame is None:
                frame_diff = np.zeros_like(gray_frame, dtype=np.float32)
            else:
                frame_diff = cv2.absdiff(prev_gray_frame, gray_frame)
                # frame_diff = cv2.absdiff(prev_gray_frame, gray_frame) / 255.0
            
            prev_gray_frame = gray_frame
            
            frames.append(frame_rgb)
            frame_diffs.append(frame_diff)
            labels.append(label)  # 0 for still, 1 for moving
        # print(len(labels))
        video_cap.release()

    return np.array(frames), np.array(frame_diffs), np.array(labels)

In [5]:
# Reshaping to 224 x 224 x 4 for the convolutional model
def combine_frames_and_diffs(frames, frame_diffs):
    # frame_diffs dimension
    # print(frame_diffs.shape)
    frame_diffs_expanded = np.expand_dims(frame_diffs, axis=-1) # Add an extra dimension
    # frame_diffs_expanded dimension
    # print(frame_diffs_expanded.shape)
    combined_input = np.concatenate([frames, frame_diffs_expanded], axis=-1) # Concatenate along the last axis
    # combined_input dimension
    # print(combined_input.shape)
    
    # Visualize the difference
    # cv2.imwrite('img_still.png', combined_input[10])
    return combined_input

In [12]:
# Generate the frames and frame_diffs array for still
still_frames, still_diff, still_labels = process_videos(default_paths, 0)
test_still_frames, test_still_diff, test_still_labels = process_videos(test_default_paths, 0)
# print(still_frames.shape)
falling_frames, falling_diff, falling_labels = process_videos(falling_paths, 1)
test_falling_frames, test_falling_diff, test_falling_labels = process_videos(test_falling_paths, 1)
# print(falling_frames.shape)

# Combine them
concatenate_frames = np.concatenate([still_frames, falling_frames], axis = 0)
# print(concatenate_frames.shape)
concatenate_diff =  np.concatenate([still_diff, falling_diff], axis = 0)
# print(concatenate_diff.shape)
concatenate_labels =  np.concatenate([still_labels, falling_labels], axis = 0)
# print(concatenate_labels.shape)

# Create 224x224x4 shape for the model
combined_input = combine_frames_and_diffs(concatenate_frames, concatenate_diff)
# print(combined_input)

# Combine them
test_concatenate_frames = np.concatenate([test_still_frames, test_falling_frames], axis = 0)
test_concatenate_diff =  np.concatenate([test_still_diff, test_falling_diff], axis = 0)
test_concatenate_labels =  np.concatenate([test_still_labels, test_falling_labels], axis = 0)
# print(test_concatenate_frames.shape)
# Create 224x224x4 shape for the model
test_combined_input = combine_frames_and_diffs(test_concatenate_frames, test_concatenate_diff)
print(combined_input.shape)
print(test_combined_input.shape)
# Shuffle the data if needed
indices = np.arange(combined_input.shape[0])
np.random.shuffle(indices)
combined_input = combined_input[indices]
concatenate_labels = concatenate_labels[indices]

#data augmentation 
data_augmentation = tf.keras.Sequential([
    # layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
    layers.RandomContrast(factor=0.2)

])


(1080, 224, 224, 4)
(1015, 224, 224, 4)


In [13]:
# Create a TensorFlow dataset
with tf.device('/cpu:0'):
    dataset = tf.data.Dataset.from_tensor_slices((combined_input, concatenate_labels))
    dataset = dataset.batch(16).prefetch(buffer_size=tf.data.AUTOTUNE)
    test_dataset = tf.data.Dataset.from_tensor_slices((test_combined_input, test_concatenate_labels))
    test_dataset = test_dataset.batch(16).prefetch(buffer_size=tf.data.AUTOTUNE)

# Split the dataset into training and validation
train_dataset = dataset.take(int(0.8 * len(dataset)))
val_dataset = dataset.skip(int(0.8 * len(dataset)))

# dataset = tf.data.Dataset.from_tensor_slices((combined_input, concatenate_labels))
# Batch and prefetch if needed
# dataset = dataset.batch(16).prefetch(buffer_size=tf.data.AUTOTUNE)

#### Create the model

In [14]:
def create_model():
    combined_input = Input(shape=(224, 224, 4), name='combined_input')

    # Apply data augmentation here
    x = data_augmentation(combined_input)

    # Convolutional layers for image processing
    # Convolutional layers are useful for learning spatial hierarchies and detecting features 
    x = Conv2D(32, (3, 3), activation='relu')(x) #  Convolutional layers extract spatial features from the image
    x = MaxPooling2D((2, 2))(x) #  Max-pooling layers reduce the size of the feature maps.
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    
    # Fully connected layers: These layers learn to make decisions based on the features extracted by the convolutional layers
    # x = Flatten()(x) # This line flattens the 2D feature maps into a 1D vector
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    x = Dense(64, activation='relu')(x)
    output = Dense(1, activation='sigmoid')(x)  # Single output with sigmoid activation

    model = Model(inputs=combined_input, outputs=output)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])  # Binary cross-entropy loss
    
    return model
    
model = create_model()
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 combined_input (InputLayer)  [(None, 224, 224, 4)]    0         
                                                                 
 sequential_1 (Sequential)   (None, 224, 224, 4)       0         
                                                                 
 conv2d_3 (Conv2D)           (None, 222, 222, 32)      1184      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 111, 111, 32)     0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                       

In [15]:
model.fit(train_dataset, validation_data=val_dataset, epochs=5)

Epoch 1/5
54/54 [==============================] - 3s 40ms/step - loss: 0.7906 - accuracy: 0.7141 - val_loss: 0.4825 - val_accuracy: 0.8333
Epoch 2/5
54/54 [==============================] - 2s 39ms/step - loss: 0.3707 - accuracy: 0.8507 - val_loss: 0.2282 - val_accuracy: 0.9306
Epoch 3/5
54/54 [==============================] - 2s 39ms/step - loss: 0.3426 - accuracy: 0.8738 - val_loss: 0.2141 - val_accuracy: 0.9398
Epoch 4/5
54/54 [==============================] - 2s 39ms/step - loss: 0.2935 - accuracy: 0.8854 - val_loss: 0.2182 - val_accuracy: 0.9491
Epoch 5/5
54/54 [==============================] - 2s 39ms/step - loss: 0.2982 - accuracy: 0.8889 - val_loss: 0.2227 - val_accuracy: 0.9398


#### Save the model

In [16]:
model.evaluate(test_dataset)
# model.save("m1")

64/64 [==============================] - 1s 11ms/step - loss: 0.4543 - accuracy: 0.7714


[0.4542892277240753, 0.7714285850524902]

#### Quantize and save as tflite

In [11]:
def representative_dataset():
    for data, label in tf.data.Dataset.from_tensor_slices((combined_input, concatenate_labels)).batch(1).take(100):
        yield [tf.dtypes.cast(data, tf.float32)]

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8
tflite_quant_m1 = converter.convert()

# Save the quantized model 1
with open('m1_quant.tflite', 'wb') as f: f.write(tflite_quant_m1)

INFO:tensorflow:Assets written to: /tmp/tmprhk8dmwg/assets


/home/sylvia/.local/lib/python3.8/site-packages/tensorflow/lite/python/convert.py:746: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
2023-08-10 10:32:08.581405: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2023-08-10 10:32:08.581426: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
2023-08-10 10:32:08.581972: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmprhk8dmwg
2023-08-10 10:32:08.584459: I tensorflow/cc/saved_model/reader.cc:78] Reading meta graph with tags { serve }
2023-08-10 10:32:08.584471: I tensorflow/cc/saved_model/reader.cc:119] Reading SavedModel debug info (if present) from: /tmp/tmprhk8dmwg
2023-08-10 10:32:08.593697: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2023-08-10 10:32:08.64136

#### Pass in video for model 1 to predict to generate data for model 2

In [17]:
# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="m1_quant.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
print('Input shape:', input_details[0]['shape'])
output_details = interpreter.get_output_details()
print('Output shape:', output_details[0]['shape'])

# [1,1] u get [[1]], instead of [1]
# idk why it wraps an extra bracket but it just does

Input shape: [  1 224 224   4]
Output shape: [1 1]


#### Predict with model 1 and save the results into an array

In [18]:
def process_single_vid(path):
    frames = []
    frame_diffs = []
    video_cap = cv2.VideoCapture(path)
    
    prev_gray_frame = None
    
    while video_cap.isOpened():
        ret, frame = video_cap.read()
        
        if not ret:
            break
        
        # Resize and convert frame to RGB
        frame_resized = cv2.resize(frame, (224, 224))
        frame_rgb = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2RGB)
        
        # Convert frame to grayscale
        gray_frame = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2GRAY)

        if prev_gray_frame is None:
            frame_diff = np.zeros_like(gray_frame, dtype=np.float32)
        else:
            frame_diff = cv2.absdiff(prev_gray_frame, gray_frame)
            # frame_diff = cv2.absdiff(prev_gray_frame, gray_frame) / 255.0
        
        prev_gray_frame = gray_frame
        
        frames.append(frame_rgb)
        frame_diffs.append(frame_diff)

    video_cap.release()
    return np.array(frames), np.array(frame_diffs)

def predict_single_vid(interpreter, input_data):
    # Get the input and output details
    input_details = interpreter.get_input_details()[0]
    output_details = interpreter.get_output_details()[0]

    # Initialize an empty list to store the results
    results = []

    # Iterate over each instance in the input data
    for instance in input_data:
        # Set the tensor to point to the input data to be inferred
        interpreter.set_tensor(input_details['index'], np.expand_dims(instance, axis=0))

        # Run the computation
        interpreter.invoke()

        # Get the output tensor
        output = interpreter.get_tensor(output_details['index'])

        # Append the output to the results list
        results.append(output)

    # Convert the results list to an array and return it
    return np.array(results)

def add_noise(data, noise_level=5):
    # Convert to float
    float_data = data.astype(np.float32)
    
    # Generate random noise
    noise = np.random.uniform(-noise_level, noise_level, data.shape)
    
    # Add the noise to the data
    noisy_data = float_data + noise
    
    # Clip values to uint8 range and convert back to uint8
    noisy_data = np.clip(noisy_data, 0, 255).astype(np.uint8)
    
    return noisy_data

In [19]:
result_8 = []
labels = []
for vid in falling_paths:
    vid_frames, vid_diffs = process_single_vid(vid)
    # reshape and combine inputs
    combined = combine_frames_and_diffs(vid_frames, vid_diffs)
    # convert datatype from float 32 to uint8
    combined = combined.astype(np.uint8)
    # print(combined)
    # predict_single_vid
    res = predict_single_vid(interpreter, combined)
    # save results to array in sliding window fashion 8 at a time
    for i in range(len(res) - 8 + 1):
        window = res[i:i+8]
        result_8.append(window)
        # print(result_8)
        labels.append(1)

for vid in default_paths:
    vid_frames, vid_diffs = process_single_vid(vid)
    # reshape and combine inputs
    combined = combine_frames_and_diffs(vid_frames, vid_diffs)
    # convert datatype from float 32 to uint8
    combined = combined.astype(np.uint8)
    # predict_single_vid
    res = predict_single_vid(interpreter, combined)
    # save results to array in sliding window fashion 8 at a time
    for i in range(len(res) - 8 + 1):
        window = res[i:i+8]
        result_8.append(window)
        labels.append(0)

In [20]:

result_8 = np.array(result_8)
labels = np.array(labels)

result_8 = result_8.reshape(len(result_8), 8)
noisy_data = add_noise(result_8)
doubled_data = np.vstack((result_8, noisy_data))
augmented_labels = labels
doubled_labels = np.hstack((labels, augmented_labels))

print(doubled_data.shape)
print(doubled_labels.shape)

np.savetxt('model2data_augmented.txt', doubled_data, fmt='%d', delimiter=' ')
np.savetxt('model2labels_augmented.txt', doubled_labels, fmt='%d', delimiter=' ')

(1740, 8)
(1740,)


In [16]:
X = np.loadtxt('model2data_augmented.txt', dtype=np.float32, delimiter=' ')
X = X/255.0

print(X.shape)
Y = np.loadtxt('model2labels_augmented.txt', dtype=np.float32, delimiter=' ')
# print(Y.shape)

# fall_X = np.loadtxt('./../K-optimized1-training/model1_output_fall.txt', dtype=np.uint8, delimiter=' ')
# default_X = np.loadtxt('./../K-optimized1-training/model1_output_default.txt', dtype=np.uint8, delimiter=' ')
# X = np.concatenate((fall_X, default_X))

# fall_Y = np.ones(len(fall_X))
# default_Y = np.zeros(len(default_X))

# Y = np.concatenate((fall_Y, default_Y))
# print(Y.shape)

# Create the dataset
dataset = tf.data.Dataset.from_tensor_slices((X, Y))
dataset = dataset.batch(16).prefetch(buffer_size=tf.data.AUTOTUNE)

(1740, 8)


#### Prepare testing data

In [47]:
test_result_8 = []
test_labels = []

for vid in test_falling_paths:
    test_vid_frames, test_vid_diffs = process_single_vid(vid)
    # reshape and combine inputs
    test_combined = combine_frames_and_diffs(test_vid_frames, test_vid_diffs)
    # convert datatype from float 32 to uint8
    test_combined = test_combined.astype(np.uint8)
    # print(combined)
    # predict_single_vid
    test_res = predict_single_vid(interpreter, test_combined)
    # save results to array in sliding window fashion 8 at a time
    for i in range(len(test_res) - 8 + 1):
        window = test_res[i:i+8]
        test_result_8.append(window)
        test_labels.append(1)

for vid in test_default_paths:
    test_vid_frames, test_vid_diffs = process_single_vid(vid)
    # reshape and combine inputs
    test_combined = combine_frames_and_diffs(test_vid_frames, test_vid_diffs)
    # convert datatype from float 32 to uint8
    test_combined = test_combined.astype(np.uint8)
    # predict_single_vid
    test_res = predict_single_vid(interpreter, test_combined)
    # save results to array in sliding window fashion 8 at a time
    for i in range(len(test_res) - 8 + 1):
        window = test_res[i:i+8]
        test_result_8.append(window)
        test_labels.append(0)

test_result_8 = np.array(test_result_8)
test_labels = np.array(test_labels)


In [48]:
print(test_result_8.shape)
print(test_labels.shape)
test_result_8 = test_result_8.reshape(len(test_result_8), 8)
np.savetxt('model2data_test.txt', test_result_8, fmt='%d', delimiter=' ')
np.savetxt('model2labels_test.txt', test_labels, fmt='%d', delimiter=' ')
print(test_result_8.shape)

(805, 8, 1, 1)
(805,)
(805, 8)


In [17]:
test_X = np.loadtxt('model2data_test.txt', dtype=np.float32, delimiter=' ')
test_X = test_X/255.0
print(test_X.shape)
test_Y = np.loadtxt('model2labels_test.txt', dtype=np.float32, delimiter=' ')
print(test_Y.shape)

# test_fall_X = np.loadtxt('./../K-optimized1-training/model1_output_fall_test.txt', dtype=np.uint8, delimiter=' ')
# test_default_X = np.loadtxt('./../K-optimized1-training/model1_output_default_test.txt', dtype=np.uint8, delimiter=' ')
# test_X = np.concatenate((test_fall_X, test_default_X))

# test_fall_Y = np.ones(len(test_fall_X))
# test_default_Y = np.zeros(len(test_default_X))

# test_Y = np.concatenate((test_fall_Y, test_default_Y))
# print(test_Y.shape)

# Create the dataset
test_dataset = tf.data.Dataset.from_tensor_slices((test_X, test_Y))
test_dataset = test_dataset.batch(16).prefetch(buffer_size=tf.data.AUTOTUNE)

(805, 8)
(805,)


#### Create model 2

In [24]:
def create_model_2():
    input_shape = (8, )  # Input shape for the sequence of 8 uint8 values
    
    # Define the model
    input_layer = Input(shape=input_shape, name='input_layer')
    
    # Several dense layers to extract features and relationships
    x = Dense(64, activation='relu')(input_layer)
    x = Dropout(0.5)(x)
    x = Dense(32, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(16, activation='relu')(x)
    
    # Output layer
    output = Dense(1, activation='sigmoid')(x)

    # Create and compile the model
    model = Model(inputs=input_layer, outputs=output)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

model_2 = create_model_2()
model_2.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 8)]               0         
                                                                 
 dense_8 (Dense)             (None, 64)                576       
                                                                 
 dropout_4 (Dropout)         (None, 64)                0         
                                                                 
 dense_9 (Dense)             (None, 32)                2080      
                                                                 
 dropout_5 (Dropout)         (None, 32)                0         
                                                                 
 dense_10 (Dense)            (None, 16)                528       
                                                                 
 dense_11 (Dense)            (None, 1)                 17  

#### Prepare Tensorflow dataset

In [25]:
model_2.fit(dataset, epochs=15)

Epoch 1/15
109/109 [==============================] - 0s 2ms/step - loss: 0.5715 - accuracy: 0.5333
Epoch 2/15
109/109 [==============================] - 0s 2ms/step - loss: 0.4346 - accuracy: 0.8770
Epoch 3/15
109/109 [==============================] - 0s 2ms/step - loss: 0.3165 - accuracy: 0.9529
Epoch 4/15
109/109 [==============================] - 0s 2ms/step - loss: 0.1935 - accuracy: 0.9621
Epoch 5/15
109/109 [==============================] - 0s 2ms/step - loss: 0.1433 - accuracy: 0.9672
Epoch 6/15
109/109 [==============================] - 0s 2ms/step - loss: 0.1165 - accuracy: 0.9684
Epoch 7/15
109/109 [==============================] - 1s 5ms/step - loss: 0.1155 - accuracy: 0.9724
Epoch 8/15
109/109 [==============================] - 0s 4ms/step - loss: 0.1148 - accuracy: 0.9736
Epoch 9/15
109/109 [==============================] - 0s 2ms/step - loss: 0.1043 - accuracy: 0.9776
Epoch 10/15
109/109 [==============================] - 0s 2ms/step - loss: 0.0979 - accuracy: 0.9759

In [26]:
model_2.evaluate(test_dataset)

51/51 [==============================] - 0s 996us/step - loss: 0.8359 - accuracy: 0.8932


[0.8359243273735046, 0.8931676745414734]

In [27]:
print(X.dtype)
def representative_dataset_2():
    for i in range(len(X)):
        sample = X[i:i+1].astype(np.float32)  # Extract and cast in one line
        yield [sample]

converter = tf.lite.TFLiteConverter.from_keras_model(model_2)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset_2
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

tflite_model = converter.convert()

# Save the model to disk
with open('m2_quant.tflite', 'wb') as f: f.write(tflite_model)

float32
INFO:tensorflow:Assets written to: /tmp/tmp0q_s911v/assets


INFO:tensorflow:Assets written to: /tmp/tmp0q_s911v/assets
/home/sylvia/.local/lib/python3.8/site-packages/tensorflow/lite/python/convert.py:746: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
2023-08-10 13:58:37.414938: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2023-08-10 13:58:37.414960: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
2023-08-10 13:58:37.415137: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmp0q_s911v
2023-08-10 13:58:37.416346: I tensorflow/cc/saved_model/reader.cc:78] Reading meta graph with tags { serve }
2023-08-10 13:58:37.416358: I tensorflow/cc/saved_model/reader.cc:119] Reading SavedModel debug info (if present) from: /tmp/tmp0q_s911v
2023-08-10 13:58:37.420355: I tensorflow/cc/saved_model/loader.cc:

ValueError: Cannot set tensor: Got value of type INT8 but expected type FLOAT32 for input 0, name: serving_default_input_layer:0 

#### Checking input / output of model

In [13]:
# Load TFLite model and allocate tensors.
interpreter_2 = tf.lite.Interpreter(model_path="m2_quant.tflite")
interpreter_2.allocate_tensors()

input_details = interpreter_2.get_input_details()
print('Input shape:', input_details[0]['shape'])
print('Input type:', input_details[0]['dtype'])
output_details = interpreter_2.get_output_details()
print('Output shape:', output_details[0]['shape'])
print('Output type:', output_details[0]['dtype'])

Input shape: [1 8]
Input type: <class 'numpy.uint8'>
Output shape: [1 1]
Output type: <class 'numpy.uint8'>
